# <span style="color:red">**exercise 3**</span>
![Python logo](./python_logo.gif)
---

### submited by:
- Name: Shahar Asher
- Id: 209305408
- Email adress: shaharas@edu.hac.ac.il
- Date: 12/04/2024

### Operation system: Windows 11
### Python version: 3.11.5 (Using anaconda)
### IDE: Visual Studio Code
### External libraries: numpy, matplotlib
---

In [246]:
# imports
import pandas as pd
from dataclasses import dataclass, make_dataclass

## Q.1: 
- 

In [247]:
import re

def make_valid_identifier(value:str)->str:
    # Replace invalid characters with underscores
    identifier:str = re.sub(r'\W|^(?=\d)', '_', value)

    # Check if the identifier starts with a digit, if so, add an underscore
    identifier = 'nV_' + identifier
    while(identifier.endswith('_')):
        identifier = identifier[:-1]
    
    for i in range(0, len(identifier)):
        if i+1 < len(identifier):
            if identifier[i] == '_' and identifier[i+1] == '_':
                identifier = identifier[:i] + identifier[i+1:]

    return identifier

In [248]:
def make_DF(data:dict)->pd.DataFrame:
    df:pd.DataFrame = pd.DataFrame(data)
    return df

In [249]:
def create_instances(df:pd.DataFrame)->dict:

    # Create a dataclass with fields corresponding to dataframe columns
    fields = [(make_valid_identifier(str(col)), type(df[col].iloc[0])) for col in df.columns]
    # Generate a dictionary of dataclass instances with unique names
    DataClass = make_dataclass("DynamicClass", fields)
    instances = {}
    for i, row in enumerate(df.itertuples(index=False)):
        instance_name = chr(ord('a') + i)*2
        instances[instance_name] = DataClass(*row)
    
    return instances

In [250]:
data:dict = {1: [1,4,6,3,10],
        'name': ['a', 'b','c','d', 'e'],
        'sur name': ['Q','B','!','9','A'],
        'Payment (US₪)': [12.3, 4.2, -9.12, 0.0, 0.14]
        }

df:pd.DataFrame = make_DF(data)
instances:dict = create_instances(df)

# Print the instances
for name, instance in instances.items():
        print(f"{name} = {instance}")

aa = DynamicClass(nV_1=1, nV_name='a', nV_sur_name='Q', nV_Payment_US=12.3)
bb = DynamicClass(nV_1=4, nV_name='b', nV_sur_name='B', nV_Payment_US=4.2)
cc = DynamicClass(nV_1=6, nV_name='c', nV_sur_name='!', nV_Payment_US=-9.12)
dd = DynamicClass(nV_1=3, nV_name='d', nV_sur_name='9', nV_Payment_US=0.0)
ee = DynamicClass(nV_1=10, nV_name='e', nV_sur_name='A', nV_Payment_US=0.14)
